Introducing the library "Shapely"

In [1]:
#likely you already have Shapely if you installed geopandas.

#geopandas dataframes have their geometries as shapely geometry objects...that's why the 2 libraries behave well together.

#test if you have it:

import shapely

In [2]:
import geopandas
from shapely.geometry import Point


In [3]:

#we are going to make a shapefile using geopandas, with points made by shapely

my_crs = 26912  #our spatial ref

#empty point array
my_points = []

#make point
new_point = Point([495809.00, 6295998.50])

#add point to array
my_points.append(new_point)


#make and add in one line
my_points.append(Point([495819.00, 6295999.50]))

#and a few more...sorta random
my_points.append(Point([495821.00, 6296000.0]))
my_points.append(Point([495822.00, 6295992.25]))
my_points.append(Point([495815.00, 6295994.0]))
my_points.append(Point([495819.00, 6295990.1]))


In [4]:
#make a geopandas data frame

new_gdf = geopandas.GeoDataFrame( None, geometry=my_points, crs=my_crs )
#the "None" varible is because we do not have any other columns.

#inspect it
print(new_gdf.head())

#we will also make an "id" field for the data frame...have a number from 0 to length -1 of our points.
new_gdf["id"] = range(len(my_points))

#inspect it again
print(new_gdf.head(4)) #the default is 5 for rows, but it can be changed...in this case, 4



                         geometry
0  POINT (495809.000 6295998.500)
1  POINT (495819.000 6295999.500)
2  POINT (495821.000 6296000.000)
3  POINT (495822.000 6295992.250)
4  POINT (495815.000 6295994.000)
                         geometry  id
0  POINT (495809.000 6295998.500)   0
1  POINT (495819.000 6295999.500)   1
2  POINT (495821.000 6296000.000)   2
3  POINT (495822.000 6295992.250)   3


In [5]:
#happy? save it.
new_gdf.to_file('my_points.shp')


**Bonus Round**

*We have a question::*

"If I want to calculate the distance from all points inside the gdf to a feature (a point for example) then add that as a new field. How can I do that?"


In [6]:

#you may get your feature from some other source (shapefile etc)
# but for our example, let's just make a point

my_test_point = Point([495895.00, 6295984.0])


In [7]:
#shapely has a very cool method that can get the distance from one geometry to another
#no matter if they are points, linestrings, or polygons
# geom.distance(geom2)
#let us test it out

print(my_test_point.distance(my_test_point))


0.0


In [8]:
#so far so good...the distance from a point to itself *should* be zero!
#check distance to the first point in our point array
print(my_test_point.distance(my_points[0]))

87.21381771256203


In [9]:
#ok, testing done. let's get the distances from every point in our shapefile to this test point

#first, make new column to put the values in:
new_gdf['d2test'] = 0

print(new_gdf.head(3))



                         geometry  id  d2test
0  POINT (495809.000 6295998.500)   0       0
1  POINT (495819.000 6295999.500)   1       0
2  POINT (495821.000 6296000.000)   2       0


In [10]:
#there are several ways you can fill this in. I will do a couple.

#from dataframe, get geometries:
geoms = new_gdf.geometry

#make array to put values in
dvalues = []

#loop
for g in geoms:
    dvalues.append(my_test_point.distance(g))

#and assign the dataframe column to be dvalues
new_gdf['d2test'] = dvalues

print(new_gdf.head())



                         geometry  id     d2test
0  POINT (495809.000 6295998.500)   0  87.213818
1  POINT (495819.000 6295999.500)   1  77.564489
2  POINT (495821.000 6296000.000)   2  75.709973
3  POINT (495822.000 6295992.250)   3  73.464702
4  POINT (495815.000 6295994.000)   4  80.622577


In [12]:
#second method. populating a new column with data from another...using apply

new_gdf['d2test2'] = new_gdf.apply(lambda x: my_test_point.distance(x.geometry), axis=1)

print(new_gdf.head())

                         geometry  id     d2test    d2test2
0  POINT (495809.000 6295998.500)   0  87.213818  87.213818
1  POINT (495819.000 6295999.500)   1  77.564489  77.564489
2  POINT (495821.000 6296000.000)   2  75.709973  75.709973
3  POINT (495822.000 6295992.250)   3  73.464702  73.464702
4  POINT (495815.000 6295994.000)   4  80.622577  80.622577
